<a href="https://colab.research.google.com/github/F-Bafti/AI-Agents-and-Agentic-AI/blob/master/DirAgent_with_Function_Calling_Coursera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using LLM Function Calling for AI-Agent Interaction
One of the most challenging aspects of integrating AI agents with tool execution is ensuring that the model consistently produces structured output that can be parsed correctly. Traditionally, developers would attempt to engineer prompts to make the model output well-formed JSON, but this approach is unreliable—models can introduce variations, omit required fields, or output unstructured text that breaks parsing logic.

To solve this, most LLMs offer function calling APIs that guarantee structured execution. Instead of treating function execution as a free-form text generation task, function calling APIs allow us to explicitly define the tools available to the model using JSON Schema. The model then decides when and how to call these functions, ensuring structured and predictable responses.

When using function calling, the model returns either:

1 - **A function call** that includes the tool name and arguments as structured JSON.


2- **A standard text response** if the model decides a function is unnecessary.


This approach removes the need for manual prompt engineering to enforce structured output and allows the agent to focus on decision-making rather than syntax compliance.

In [1]:
!pip install langchain-community
!pip uninstall cohere langchain-cohere -y

!pip install cohere>=5.0.0
!pip install langchain-cohere

!pip install cohere==5.11.0 langchain-cohere==0.3.0


import os, json
from google.colab import userdata
from langchain_cohere import ChatCohere
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage

api_key = userdata.get('COHERE_API_KEY')
os.environ['COHERE_API_KEY'] = api_key

Found existing installation: cohere 5.16.1
Uninstalling cohere-5.16.1:
  Successfully uninstalled cohere-5.16.1
Found existing installation: langchain-cohere 0.4.4
Uninstalling langchain-cohere-0.4.4:
  Successfully uninstalled langchain-cohere-0.4.4
  Using cached langchain_cohere-0.4.4-py3-none-any.whl.metadata (6.6 kB)
Using cached langchain_cohere-0.4.4-py3-none-any.whl (42 kB)
  Using cached cohere-5.11.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached langchain_cohere-0.3.0-py3-none-any.whl.metadata (6.7 kB)
Using cached cohere-5.11.0-py3-none-any.whl (249 kB)
Using cached langchain_cohere-0.3.0-py3-none-any.whl (43 kB)
  Attempting uninstall: cohere
    Found existing installation: cohere 5.16.1
    Uninstalling cohere-5.16.1:
      Successfully uninstalled cohere-5.16.1
  Attempting uninstall: langchain-cohere
    Found existing installation: langchain-cohere 0.4.4
    Uninstalling langchain-cohere-0.4.4:
      Successfully uninstalled langchain-cohere-0.4.4
sagemaker.config 

# Function to Generate Response from LLM

In [151]:
from typing import List
from langchain_core.tools import tool

@tool
def list_files(directory: str = ".") -> List[str]:
    """List files in the specified directory (defaults to current directory)."""
    try:
        return os.listdir(directory)
    except FileNotFoundError:
        return [f"Error: Directory '{directory}' not found."]
    except Exception as e:
        return [f"Error: {str(e)}"]

@tool
def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def terminate_conversation(message: str = "Task completed successfully.") -> str:
    """Terminate the conversation with a final message to the user when no further action is required."""
    return f"CONVERSATION_TERMINATED: {message}"


# Define the tools list to bind to the response
tools = [list_files, read_file, terminate_conversation]

# Define the tool dics to apply the tool later
tool_functions = {
    "list_files": list_files.func,
    "read_file": read_file.func,
    "terminate_conversation": terminate_conversation.func
}

# Then your function
def generate_response(messages: List) -> AIMessage:
    """Call LLM to get response using LangChain ChatCohere"""
    chat = ChatCohere(
        model="command-r",
        max_tokens=1024,
    )
    # Bind tools to the chat model
    chat_with_tools = chat.bind_tools(tools)
    response = chat_with_tools.invoke(messages)
    return response

# Setup Function Calling

In [152]:
# Our rules are simplified since we don't have to worry about getting a specific output format
agent_rules = [SystemMessage(
    content="""
You are an AI agent with access to file system tools.

Available tools:
- list_files: Use this to see what files exist in a directory
- read_file: Use this to read the contents of a specific file
- terminate_conversation: Use this when the user says "terminate", "done", "finished", or when the task is complete

When a user says "terminate" or indicates they're done, you MUST use the terminate_conversation tool.
Always use tools when they are relevant to the user's request.
""")]

# Interacting with LLM

In [153]:
def run_agent():
    memory = []

    while True:
        # Get user input
        user_task = input("What would you like me to do? (or 'quit' to exit): ")

        if user_task.lower() in ['quit', 'exit', 'stop']:
            print("Goodbye!")
            break

        # Add user message to memory
        memory.append(HumanMessage(content=user_task))

        # Run the agent for this specific request
        max_iterations = 10
        iterations = 0

        while iterations < max_iterations:
            iterations += 1
            messages = agent_rules + memory
            response = generate_response(messages)

            if response.tool_calls:
                tool_call = response.tool_calls[0]
                tool_name = tool_call["name"]
                tool_args = tool_call["args"]

                if tool_name == "terminate_conversation":
                    termination_message = tool_args.get('message', 'Task completed successfully.')
                    print(f"{termination_message}")

                    # Add the termination to memory and break the inner loop
                    memory.append(response)
                    break

                elif tool_name in tool_functions:
                    try:
                        result = tool_functions[tool_name](**tool_args)
                        result_dict = {"result": result}
                    except Exception as e:
                        result_dict = {"error": f"Error executing {tool_name}: {str(e)}"}
                else:
                    result_dict = {"error": f"Unknown tool: {tool_name}"}

                print(f"Executing: {tool_name} with args {tool_args}")
                print(f"Result: {result_dict}")

                # Add messages to memory
                memory.append(response)
                memory.append(ToolMessage(
                    content=json.dumps(result_dict),
                    tool_call_id=tool_call.get("id", "default_id")
                ))

            else:
                # No tool calls, just a regular response
                print(f"Response: {response.content}")
                memory.append(response)
                break

        if iterations >= max_iterations:
            print("Maximum iterations reached for this request.")

        print()  # Add a blank line for readability

# Run the agent
run_agent()

What would you like me to do? (or 'quit' to exit): what is in this folder
Executing: list_files with args {}
Result: {'result': ['.config', '=5.0.0', 'sample_data']}
Response: The folder contains three items: .config, =5.0.0, and sample_data.

What would you like me to do? (or 'quit' to exit): what is in sample_data folder
Executing: list_files with args {'directory': 'sample_data'}
Result: {'result': ['README.md', 'anscombe.json', 'mnist_train_small.csv', 'california_housing_train.csv', 'california_housing_test.csv', 'mnist_test.csv']}
Response: There are six files in the sample_data folder:
- README.md
- anscombe.json
- mnist_train_small.csv
- california_housing_train.csv
- california_housing_test.csv
- mnist_test.csv

What would you like me to do? (or 'quit' to exit): can you read README.md
Executing: read_file with args {'file_name': 'sample_data/README.md'}
Result: {'result': "This directory includes a few sample datasets to get you started.\n\n*   `california_housing_data*.csv` i